In [1]:
import pandas as pd
import mne as mne
import os 
import time
import numpy as np
import matplotlib.pyplot as plt
import joblib
import constants
from IPython.utils import io
import time
import sys
import yasa
from scipy.signal import welch

from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GroupShuffleSplit


from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.ensemble import AdaBoostClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import GroupKFold
from sklearn.metrics import accuracy_score


#Import my modules
import format_eeg_data
import constants
import eeg_stat_ts
import run_expts

#TS Fresh Parameter Settings
from tsfresh.feature_extraction import ComprehensiveFCParameters
from tsfresh.feature_extraction import MinimalFCParameters
from tsfresh.feature_extraction import EfficientFCParameters
from tsfresh.feature_extraction import extract_features

/user/home/ko20929/.conda/envs/sktime_latest/lib/python3.11/site-packages/antropy/fractal.py:197: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @jit((types.Array(types.float64, 1, "C", readonly=True), types.int32))


In [5]:
data_dict = {}
for data_type in ['N1','REM', 'N2', 'N3', 'Wake']:
    data_dict[data_type] = {}
    for TS_Fresh_setting in ['Minimal', 'Efficient']:
        
        # Load the features 
        load_path = '/user/home/ko20929/work/RBD_using_custom_package/Data/freq_6_second_files/'
        load_path_bpw = '/user/home/ko20929/work/RBD_using_custom_package/Data/freq_6_second_files/'
        load_path_max_freqs = '/user/home/ko20929/work/RBD_using_custom_package/Execute New Experiments/Baseline_Extensions/Gen_New_Features/generated_feats/'
        
        loaded_data = {}
        
        X_y_groups = {}
            
        X_bpw = pd.read_hdf(load_path_bpw + data_type +  'six_second_freq_df.h5', key='df', mode='r')
        y = pd.read_hdf(load_path + data_type +  '_y.h5', key='df', mode='r') 
        groups = pd.read_hdf(load_path + data_type +  '_groups.h5', key='df', mode='r')
        X_bpw, y , groups = X_bpw.reset_index(drop = True) , y.reset_index(drop = True) , groups.reset_index(drop = True)
        
        X_max_freqs = pd.read_hdf(load_path_max_freqs + data_type +  'six_second_max_freq_stats_df.h5', key='df', mode='r')
        X_max_freqs = X_max_freqs.reset_index(drop = True)
        
        X = pd.concat([X_bpw , X_max_freqs], axis = 1)
        
        #Transform the X into TS_Fresh Features___
        # 1. Convert to TS_Fresh format Dataframe 
        ts_fresh_df = format_eeg_data.convert_sktime_df_to_ts_fresh_format(X, ts_cols = list(X.columns))
        
    
        # 2. Extract TS_Fresh Features from the dataframe____________
        if TS_Fresh_setting == 'Minimal':
            settings = MinimalFCParameters()
        elif TS_Fresh_setting == 'Efficient':
            settings = EfficientFCParameters()
        else:
            raise Exception('No TS Fresh Parameter Setting Set!!')
        
        extracted_ts_fresh_df = extract_features(ts_fresh_df, column_id = 'id' , column_sort = 'time', default_fc_parameters=settings)
        
        # 3. Asign extract_ts_fresh_df to the variable X
        X = extracted_ts_fresh_df.copy()
        print(len(X.columns))
        #Drop columns where all values are NA 
        X = X.dropna(axis = 1)
        print(len(X.columns))
        
        #Drop columns where all values are the same
        # Find columns where all values are the same
        same_value_columns = X.columns[X.nunique() == 1]
        # Drop columns with the same values
        X = X.drop(columns=same_value_columns)
        print(len(X.columns))
    
        #Generate a region to features dictionary - this will enable us to run expts regionally as before
        regional_features_dict = {}
        region_channel_dict = constants.region_to_channel_dict
        regions = list(region_channel_dict.keys())
        
        for region in regions:
            region_features = [col for col in X.columns if region + '_' in col]
            regional_features_dict[region] = region_features
        
        #Save first row
        X = X.iloc[:1,:]
        
        data_dict[data_type]['X_' + TS_Fresh_setting] = X
        data_dict[data_type]['regional_features_dict'] = regional_features_dict

19-Nov-23 21:51:05 | WARNING | Dependency not available for matrix_profile, this feature will be disabled!
Feature Extraction: 100%|██████████| 160/160 [00:01<00:00, 80.22it/s]


1950
1950
1948


19-Nov-23 21:51:11 | WARNING | Dependency not available for matrix_profile, this feature will be disabled!
Feature Extraction: 100%|██████████| 160/160 [04:40<00:00,  1.75s/it]


151515
76570
65942


19-Nov-23 21:56:12 | WARNING | Dependency not available for matrix_profile, this feature will be disabled!
Feature Extraction: 100%|██████████| 159/159 [00:02<00:00, 71.06it/s]


...

151515
151060
140143


19-Nov-23 22:55:31 | WARNING | Dependency not available for matrix_profile, this feature will be disabled!
Feature Extraction: 100%|██████████| 159/159 [00:02<00:00, 68.72it/s] 


1950
1950
1950


19-Nov-23 22:55:38 | WARNING | Dependency not available for matrix_profile, this feature will be disabled!
Feature Extraction: 100%|██████████| 159/159 [02:52<00:00,  1.08s/it]


151515
64440
53294


In [6]:
joblib.dump(data_dict, 'data_dict_2_region_only.pkl')

['data_dict_2_region_only.pkl']